<a href="https://colab.research.google.com/github/ak283/language-translation-using-llm-model/blob/main/Language_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The Project is devided into 2 parts:**
- Data collection
- Language translation

In [1]:
import requests as req
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

 1. **Data generation through web scraping**

The 1st step of the project is to generate data and for this web scraping of www.cars24.com has been done which is 1 of the most popular sites in india related to the used cars.

In this step:-
- Name of the car
-Model of the car
- Km run
-Fuel type
-Transmission
-Priceof the car
-Description of the car

Are some of the data collected from the site and stored in the dataframe.

In [2]:
url = "https://www.cars24.com/buy-used-cars-patna/"
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
response=req.get("https://www.cars24.com/buy-used-cars-patna/", headers= headers).text

soup=BeautifulSoup(response, "lxml")

In [3]:
year=[]
name=[]
km=[]
price=[]
fuel =[]
trans =[]
disc=[]

In [4]:
for j in soup.find_all('a', attrs={'class':"IIJDn"}):
  for i in j.find_all('h3', attrs={'class':"_11dVb"}):
    name.append(i.text.strip()[5:])
    year.append(j.text.strip()[0:4])

  for i in j.find_all('ul', attrs={'class':"_3J2G-"}):
    km.append(i.text.strip())

  for i in j.find_all('strong', attrs={'class':"_3RL-I"}):
    price.append(float(i.text.strip()[1:5]))

  for i in j.find('p'):
    disc.append(i.text.strip())
        #company.append(j.text.strip())

In [5]:
name

['Tata TIGOR XZ PLUS PETROL',
 'Hyundai Verna FLUIDIC 1.6 CRDI SX',
 'Tata Tiago XZ PLUS DUAL TONE PETROL',
 'Maruti New Wagon-R VXI (O) 1.2 AMT',
 'Ford Ecosport TITANIUM 1.5L PETROL',
 'Hyundai Creta S 1.6 PETROL',
 'Mahindra XUV300 W6 1.2 PETROL',
 'Maruti Swift Dzire VXI',
 'Hyundai i10 SPORTZ 1.2',
 'Maruti Baleno ZETA PETROL 1.2',
 'Honda Amaze 1.5L I-DTEC S OPT',
 'Hyundai NEW SANTRO SPORTZ MT',
 'Hyundai Eon MAGNA +',
 'Maruti Alto LX',
 'Maruti Baleno DELTA PETROL 1.2',
 'Maruti Vitara Brezza ZDI PLUS',
 'Datsun Redi Go 1.0 T(O) AT',
 'Maruti Swift VXI',
 'Maruti Dzire ZXI PLUS AMT',
 'Maruti Celerio X ZXI (O)']

In [6]:
for i in km:
  if 'Petrol' in i:
    fuel.append('Petrol')
  elif 'Diesel' in i:
    fuel.append('Diesel')
  else:
    fuel.append('Nan')

  if 'Manual' in i:
    trans.append('Manual')
  elif 'Automatic' in i:
    trans.append('Automatic')
  else:
    trans.append('Nan')



In [7]:
a=[]
regex = '\d+,\d+'
for i in km:
  a.append(re.findall(regex, i)[0])

In [8]:
columns={'Car':name, 'Model_yr':year, 'Km_run':a, 'Price(lakh)':price, 'Fuel': fuel, 'Transmission':trans,
             'Description':disc}



In [10]:
data = pd.DataFrame(columns)
data.head()

,Car,Model_yr,Km_run,Price(lakh),Fuel,Transmission,Description
0,Tata TIGOR XZ PLUS PETROL,2021,"22,821",6.54,Petrol,Manual,"Explore this second hand Tata TIGOR, a 2021 mo..."
1,Hyundai Verna FLUIDIC 1.6 CRDI SX,2012,"93,863",3.69,Diesel,Manual,"The 2012 Hyundai Verna FLUIDIC 1.6 CRDI SX, a ..."
2,Tata Tiago XZ PLUS DUAL TONE PETROL,2020,"43,282",4.68,Petrol,Manual,This used 2020 Tata Tiago XZ Plus Dual Tone Pe...
3,Maruti New Wagon-R VXI (O) 1.2 AMT,2020,"38,720",4.58,Petrol,Automatic,The 2020 Maruti New Wagon-R VXI (O) 1.2 AMT in...
4,Ford Ecosport TITANIUM 1.5L PETROL,2018,"70,433",5.92,Petrol,Manual,The 2018 Ford Ecosport Titanium 1.5L Petrol in...


All the above mentioned data is scraped using request and BeautifulSoup library and stored in a dataframe

In [11]:
import torch

In [12]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [13]:
translate=[]
for text in data['Description']:
  tokenizer.src_lang = "en_XX"
  encoded_en = tokenizer(text, return_tensors="pt")
  generated_tokens = model.generate(
    **encoded_en,
    forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
  )
  translate.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

In [14]:
tokenizer.src_lang = "en_XX"
encoded_hi = tokenizer(text, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
)

In [15]:
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

'पटना में 61,298 कि. मी. के साथ 2018 मारुटी सेलेरीओ एक्स ZXI (O) शहरी ड्राइविंग के लिए एक stylish और व्यावहारिक विकल्प है. एक प्रथम मालिक के रूप में, पेट्रोलियम वाहन के रूप में, यह हैंडल ट्रांसमिशन, एलॉयर चक्र, और एअरबॅग और ABS जैसे प्रमुख सुरक्षा विशेषताओं को शामिल करता है. यह नियमित सर्विसिंग के साथ अच्छी हालत में है के रूप में वर्णन किया गया है. दैनिक उपयोग के लिए एक विश्वसनीय और कुशल कार खोजने के लिए आदर्श है.'

In [16]:
data['hindi_desc']=translate

In [17]:
data.head()

,Car,Model_yr,Km_run,Price(lakh),Fuel,Transmission,Description,hindi_desc
0,Tata TIGOR XZ PLUS PETROL,2021,"22,821",6.54,Petrol,Manual,"Explore this second hand Tata TIGOR, a 2021 mo...","यह दोबारा उपयोग में लाया गया टाटा TIGOR, एक 20..."
1,Hyundai Verna FLUIDIC 1.6 CRDI SX,2012,"93,863",3.69,Diesel,Manual,"The 2012 Hyundai Verna FLUIDIC 1.6 CRDI SX, a ...",सन् 2012 में पटना में एक डीज़ल-पोरेट सैनड Hyun...
2,Tata Tiago XZ PLUS DUAL TONE PETROL,2020,"43,282",4.68,Petrol,Manual,This used 2020 Tata Tiago XZ Plus Dual Tone Pe...,यह 2020 टाटा टाियागो एक्सZ प्लस दोहरे टोन पेट्...
3,Maruti New Wagon-R VXI (O) 1.2 AMT,2020,"38,720",4.58,Petrol,Automatic,The 2020 Maruti New Wagon-R VXI (O) 1.2 AMT in...,पटना में 2020 मारुटी न्यू वागन-आर VXI (O) 1.2 ...
4,Ford Ecosport TITANIUM 1.5L PETROL,2018,"70,433",5.92,Petrol,Manual,The 2018 Ford Ecosport Titanium 1.5L Petrol in...,2018 फोर्ड ईकोपोर्ट टिटानियम 1.5 एल पेट्रोलियम...


In [27]:
pd.set_option('display.max_colwidth', None)
data[['Description','hindi_desc']].head()

,Description,hindi_desc
0,"Explore this second hand Tata TIGOR, a 2021 model with 22821 km on the clock. This used car combines efficiency with style, featuring essential amenities for a comfortable ride. Ideal for those seeking reliability and value in a pre-owned vehicle.","यह दोबारा उपयोग में लाया गया टाटा TIGOR, एक 2021 मॉडल 22821 कि. मी. घड़ी पर. यह उपयोग में लाया गया कार कुशलता और शैली को जोड़ता है, एक आरामदायक सवारी के लिए अनिवार्य सुविधाओं के साथ. एक पूर्व स्वामित्व की गाड़ी में विश्वसनीयता और मूल्य खोजने के लिए आदर्श."
1,"The 2012 Hyundai Verna FLUIDIC 1.6 CRDI SX, a diesel-powered sedan in Patna, boasts 93,773 km on the clock. As a first-owner car, it's decked with features like steering mounted controls and ABS for a blend of comfort and safety. This manual beast promises a blend of style with its sleek alloy wheels and practicality, assured with a warranty and 7-day return.","सन् 2012 में पटना में एक डीज़ल-पोरेट सैनड Hyundai Verna FLUIDIC 1.6 CRDI SX 93,773 कि. मी. घड़ी पर है. एक प्रथम स्वामित्व वाले कार के रूप में, यह आराम और सुरक्षा के मिश्रण के लिए स्टेरिंग माउन्ट नियंत्रक और ABS जैसे विशेषताओं से सज्जित है. यह हस्तचालित पशु अपने सूक्ष्म एलॉयर चक्रों और व्यावहारिकता के साथ शैली का मिश्रण की वचन देता है, जो कि गारंटी और 7 दिन की वापसी के साथ सुनिश्चित है."
2,"This used 2020 Tata Tiago XZ Plus Dual Tone Petrol, located in Patna, offers a manual transmission and has been driven 43,228 km. This pre-owned hatchback is a first-owner vehicle with a petrol engine, known for its reliability and stylish dual-tone exterior. It's a practical option for city driving with notable features.","यह 2020 टाटा टाियागो एक्सZ प्लस दोहरे टोन पेट्रोलियम का उपयोग किया गया है, जो पटना में स्थित है, एक हस्तचालित संचरण प्रदान करता है और 43,228 कि. मी. चलाया गया है. यह पूर्व स्वामित्व वाली हैचबैक एक पेट्रोलियम इंजन के साथ एक प्रथम स्वामित्व वाले वाहन है, इसकी विश्वसनीयता और stylish दोहरे टोन बाहरी के लिए जाना जाता है. यह उल्लेखनीय विशेषताओं के साथ शहरी ड्राइविंग के लिए एक व्यावहारिक विकल्प है."
3,"The 2020 Maruti New Wagon-R VXI (O) 1.2 AMT in Patna is a first-owner vehicle with a 1197 cc engine, showcasing 38,720 km. It’s packed with features like steering-mounted controls, Bluetooth compatibility, airbags, and ABS. This petrol, automatic hatchback combines convenience with safety, making it a desirable choice for city driving.","पटना में 2020 मारुटी न्यू वागन-आर VXI (O) 1.2 AMT एक 1197 सीसी इंजन वाले प्रथम स्वामित्व वाले वाहन है, जो 38,720 कि. मी. का प्रदर्शन करता है. यह स्टेरिंग-माउन्ट नियंत्रक, ब्लूटूथ संगमता, एअरबॅग, और ABS जैसे विशेषताओं से भरा हुआ है. यह पेट्रोलियम, स्वचालित हैचबैक सुविधा और सुरक्षा के साथ जोड़ता है, जिससे यह शहरी ड्राइविंग के लिए एक पसंदीदा विकल्प बनाता है."
4,"The 2018 Ford Ecosport Titanium 1.5L Petrol in Patna is a first-owner vehicle with a manual transmission and has traveled 70,349 km. It features alloy wheels, steering mounted controls, push button start, and Bluetooth compatibility. The car has 3rd-party insurance valid until January 2025, and offers warranty plans up to 12 months plus a 7-day return option.","2018 फोर्ड ईकोपोर्ट टिटानियम 1.5 एल पेट्रोलियम पटना में एक मैन्युअल ट्रांसमिशन वाले प्रथम स्वामित्व वाले वाहन है और 70,349 कि. मी. तक यात्रा कर चुका है। इसमें एलॉयड चक्का, स्टेरिंग माउन्ट किए गए नियंत्रण, धकेल बटन स्टार्ट, और ब्लूटूथ संगतता शामिल हैं। कार को जनवरी 2025 तक 3rd-party बीमा है, और 12 महीनों तक की गारंटी योजनाएं और 7 दिन की वापसी विकल्प उपलब्ध कराता है।"
